## 지진관측기록 시각화(Heat map)

In [ ]:
'''
xlsx file to 
'''
import jpype
import asposecells
jpype.startJVM()
from asposecells.api import Workbook, SaveFormat

workbook = Workbook("xlsx_file\\historical_earthquake.xlsx")
workbook.save("csv_file\\historical_earthquake.csv", SaveFormat.CSV)


workbook = Workbook("xlsx_file\\국내지진목록_1904-03-03_2023-05-25.xls")
workbook.save("csv_file\\국내지진목록_1904-03-03_2023-05-25.csv", SaveFormat.CSV)


In [ ]:
'''
2 ~ 1904 Earthquake
가중치 : 진도 -> 가속도(PGA)
'''
import pandas as pd
import numpy as np
pd.set_option('mode.chained_assignment',  None)

Rom = {'Ⅰ' : 0.035, 'Ⅱ' : 0.15, 'Ⅲ' : 0.495, 'Ⅳ' : 1.66, 'Ⅴ' : 4.71, 'Ⅵ' : 10.795, 'Ⅶ' : 23.195, 'Ⅷ' : 49.835, 'Ⅸ' : 107.075, 'Ⅹ' : 146.14, '-' : 0, '?' : 0}     # 진도 rom num : arabic num


df_his = pd.read_csv("csv_file\\historical_earthquake.csv", encoding='utf-8')
df_his = df_his.rename(columns= df_his.iloc[0])
df_his = df_his.drop(df_his.index[0])
df_his = df_his.drop(df_his.index[len(df_his['진도']) - 1])             # column, 마지막 값(없는 값) 정리

df_his['일자(양력/음력)'] = df_his['일자(양력/음력)'].astype('str')         # 양력만 추출
df_his_date = df_his['일자(양력/음력)'].str.split('/')
df_his['일자'] = df_his_date.str.get(0)

j = 1                                                                 # 진도 arabic num 변환
for i in df_his['진도']:
    if len(i) == 1:
        temp = Rom[i]
        df_his['진도'][j] = temp
    elif len(i) == 3:
        temp = i[2]
        temp = Rom[temp]
        df_his['진도'][j] = temp

    j += 1

df_his['위도/경도'] = df_his['위도/경도'].astype('str')                     # 위도 , 경도 따로 추출, 실수화
df_his = df_his[df_his['위도/경도'] != '진앙미정 / 진앙미정']
df_his_loc = df_his['위도/경도'].str.split('/')
df_his['위도'] = df_his_loc.str.get(0)
df_his['경도'] = df_his_loc.str.get(1)

df_his = df_his.drop(['일자(양력/음력)', '위도/경도', '기사(출전)', '번호', '감진반경(km)'], axis= 'columns')       # column 최종 정리

import folium
from folium import plugins
latitude, longitude = 35.9078, 127.7669                                 # 대한민국의 좌표
S_korea = folium.Map(location = [latitude, longitude], zoom_start = 7)
df_his['위도'] = df_his['위도'].astype('float')
df_his['경도'] = df_his['경도'].astype('float')
df_his['진도'] = df_his['진도'].astype('float')
S_korea.add_child(plugins.HeatMap(zip(df_his['위도'], 
                                      df_his['경도'],
                                    df_his['진도']), radius= 15))

df_his.to_csv("csv_file\\df_his.csv")   

In [ ]:
'''
1904 03 03 ~ 2023 05 25 Earthquake
가중치 : 규모 -> Energy
'''
import pandas as pd
df_rec = pd.read_csv("csv_file\\국내지진목록_1904-03-03_2023-05-25.csv", encoding= 'utf-8')
df_rec = df_rec.rename(columns= df_rec.iloc[0])
df_rec = df_rec.drop(['번호', '발생시각', '위치', '지도보기', '상세정보\n(진도 등)', '최대\n진도', '깊이(km)'], axis= 'columns')
df_rec = df_rec.drop(df_rec.index[0:1])
df_rec = df_rec.dropna(axis=0)



df_rec['위도'] = df_rec['위도'].astype('str')
rec_lat = df_rec['위도'].str.split(' ')
df_rec['위도'] = rec_lat.str.get(0)

df_rec['경도'] = df_rec['경도'].astype('str')
rec_lat = df_rec['경도'].str.split(' ')
df_rec['경도'] = rec_lat.str.get(0)

j = 1
for i in df_rec['규모']:                        # 규모 -> 에너지 환산 :  https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=gumdansan21&logNo=130135121090
    temp = (11.8 + 1.5 * float(i))
    df_rec['규모'][j] = 10 ** temp
    j += 1
    
df_rec = df_rec[df_rec['위도'] != '-']


import folium                                                            # heatmap 
from folium import plugins
latitude, longitude = 35.9078, 127.7669                                 # 대한민국의 좌표
S_korea = folium.Map(location = [latitude, longitude], zoom_start = 7)

df_rec['위도'] = df_rec['위도'].astype('float')
df_rec['경도'] = df_rec['경도'].astype('float')

S_korea.add_child(plugins.HeatMap(zip(df_rec['위도'],
                                      df_rec['경도'],
                                      df_rec['규모']), radius= 15))

## 과거 지진 데이터 규모값 있는 html crawling + 최근 지진 데이터 합치기
https://www.seismosoc.org/Publications/BSSA_html/bssa_96-3/05050-esupp/05050-esupp_1.html<br>
값이 들어있는 것은 공통적으로
```html
<td>
```
 형식을 지니고 있음<br>
 <br>
 df_his + df_rec -> new dataframe<br>
 columns : year, month, day, lat, long, ml<br>
<br>
1392 ~ 2023 data

In [3]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

html = urlopen("https://www.seismosoc.org/Publications/BSSA_html/bssa_96-3/05050-esupp/05050-esupp_1.html")
bsObject = BeautifulSoup(html, "html.parser")
data_ml = bsObject.find_all('td', class_ = False)

list_year = []      # 1
list_month = []     # 2
list_day = []       # 3
list_lat = []       # 4
list_long = []      # 5
list_ml = []        # 7

n = 0
for data in data_ml:
    k = n % 10
    if k == 1:
        list_year.append(data.get_text())
    elif k == 2:
        list_month.append(data.get_text())
    elif k == 3:
        list_day.append(data.get_text())
    elif k == 4:
        list_lat.append(data.get_text())
    elif k == 5:
        list_long.append(data.get_text())
    elif k == 7:
        list_ml.append(data.get_text())
    n += 1

raw_data = {'year' : list_year,
            'month' : list_month,
            'day' : list_day,
            'lat' : list_lat,
            'long' : list_long,
            'ml' : list_ml}
dfHisML = pd.DataFrame(raw_data)


def split_data(col):
    df_rec[col] = df_rec[col].astype('str')
    rec = df_rec[col].str.split(' ')
    df_rec[col] = rec.str.get(0)

df_rec = pd.read_csv("csv_file\\국내지진목록_1904-03-03_2023-05-25.csv", encoding= 'utf-8')
df_rec = df_rec.rename(columns= df_rec.iloc[0])
df_rec = df_rec.drop(['번호', '위치', '지도보기', '상세정보\n(진도 등)', '최대\n진도', '깊이(km)'], axis= 'columns')
df_rec = df_rec.drop(df_rec.index[0:1])
df_rec = df_rec.dropna(axis=0)

split_data('위도')
split_data('경도')
split_data('발생시각')

temp = df_rec['발생시각'].str.split('/')

list_year = temp.str.get(0).values.tolist()
list_month = temp.str.get(1).values.tolist()
list_day = temp.str.get(2).values.tolist()
list_lat = df_rec['위도'].values.tolist()
list_long = df_rec['경도'].values.tolist()
list_ml = df_rec['규모'].values.tolist()


raw_data = {'year' : list_year,
            'month' : list_month,
            'day' : list_day,
            'lat' : list_lat,
            'long' : list_long,
            'ml' : list_ml,}
dfRecML = pd.DataFrame(raw_data, dtype= object)

dfML = pd.concat([dfHisML, dfRecML], ignore_index= True)

dfML['year'] = dfML['year'].astype('int')



list_ml_erg = []
for i in dfML['ml']:                        # 규모 -> 에너지 환산 :  https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=gumdansan21&logNo=130135121090
    temp = (11.8 + 1.5 * float(i))
    list_ml_erg.append((10 ** temp) * (10 **- 19))              # 단위 : TJ
dfML['ml_erg'] = list_ml_erg
dfML = dfML.drop(dfML[dfML['year'] <= 1392].index)
dfML['lat'] = dfML['lat'].dropna()
dfML['long'] = dfML['long'].dropna()
dfML = dfML.drop(dfML[(dfML['lat'] == '-') | (dfML['long'] == '-')].index)
dfML.to_csv("csv_file\\earthquake_ML.csv")

## 좌표에 따른 시군구 위치 산출

In [5]:
import pandas as pd
import json
import geopandas as gpd
from fiona.crs import from_epsg
sigungu_geojson = 'json_file/SIGUNGU_2302.json'
dfML = pd.read_csv("csv_file\\earthquake_ML.csv")
dfML = dfML.drop(['Unnamed: 0'], axis= 'columns')

with open(sigungu_geojson,'r',encoding="utf-8") as f:
    json_data = json.load(f)

gdf_pt_geom = gpd.points_from_xy(dfML.long, dfML.lat)
eq_gpd = gpd.GeoDataFrame(dfML, geometry=gdf_pt_geom, crs=from_epsg(4326))              # dfML 좌표


sigungu_gdf = gpd.read_file(sigungu_geojson)
sigungu_gdf = gpd.GeoDataFrame(sigungu_gdf, geometry=sigungu_gdf['geometry'], crs=from_epsg(4326))          # 시군구 geodataframe

place_data = gpd.sjoin(eq_gpd, sigungu_gdf, how = "inner")
place_data['ml'] = dfML['ml_erg']
place_data['ml'] = place_data['ml'].astype('float')
sig_cd = place_data['SIG_CD'].unique()
ml_sum = []
for place in sig_cd:
    sum_ = place_data['ml'][place_data['SIG_CD'].str.contains(place)].sum()
    ml_sum.append(sum_ / 1000)

[0.7753029888532365, 0.032464728218053386, 0.06491339335592683, 0.04420496544832749, 0.01895816545047848, 0.07848155469533676, 0.013587307257181951, 0.004009617335223831, 0.03978578839740756, 0.0030892842437429146, 0.004802734837085971, 0.011748443627875539, 0.004849382492974154, 0.07224595856186784, 1.19281303274037e-05, 0.00557805897747881, 6.110288470981722e-05, 0.04146378113277513, 0.035185197081333684, 0.0007640467070992382, 0.09779412244367246, 0.009472866856308368, 0.25932604204081855, 0.02121125478214898, 0.0014698082789013438, 0.0015986942374182187, 0.00016920481570031367, 0.0055085287372026795, 0.009088107099025093, 0.008876208848181216, 0.26004968609790846, 0.7445150779009554, 0.04223528620530579, 0.009657733034630321, 0.004208431827642102, 0.009425115439552486, 0.003752322743409769, 0.0008082910627405341, 0.0041342598654745275, 0.06936327484434504, 0.001516488035767825, 0.005587230717351232, 0.0008645178779968115, 0.009641843495593444, 0.033049888354995095, 0.00013712549948